In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!ls '/content/gdrive/My Drive/computer_vision'

models	scripts  workspace


In [ ]:
# Install required packages (just for the first run)
%cd '/content/gdrive/My Drive/computer_vision'
!git clone --quiet https://github.com/tensorflow/models.git

!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

In [ ]:
# Protobuf Installation/Compilation
%cd '/content/gdrive/My Drive/computer_vision/models/research'
!protoc object_detection/protos/*.proto --python_out=.

/content/gdrive/My Drive/computer_vision/models/research


In [ ]:
# Install lost packages
!pip install tf-slim
!pip install tf-models-official
!pip install lvis

     |████████████████████████████████| 358kB 18.9MB/s 
     |████████████████████████████████| 1.1MB 19.1MB/s 
     |████████████████████████████████| 706kB 51.6MB/s 
     |████████████████████████████████| 37.6MB 79kB/s 
     |████████████████████████████████| 102kB 12.9MB/s 
     |████████████████████████████████| 174kB 39.1MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 1.2MB 45.2MB/s 
     |████████████████████████████████| 645kB 53.3MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-cp37-none-any.whl size=22245 sha256=0d0df36bf28f0d41784ea2bb60a6efacf8b3744c6f2689510bdd41139753d954
  Stored in directory: /root/.cache/pip/wheels/2e/15/f5/aa2a056d223903b52cf4870134e3a01df0c723816835dd08db
  Created wheel for seqeval: filename=seqeval-1.2.2-cp37-none-any.whl size=16172 sha256=2d1b7532b3ef5045de360c9c70320c355e8bdbaf1d8866ef1bca741fe0ca6866
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e6262147

In [ ]:
# Enabling and testing the GPU
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
# Observe TensorFlow speedup on GPU relative to CPU
%tensorflow_version 2.x
import tensorflow as tf
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()

# Run the op several times.
print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
      '(batch x height x width x channel). Sum of ten runs.')
print('CPU (s):')
cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
print(cpu_time)
print('GPU (s):')
gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
print(gpu_time)
print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
2.8247436500000163
GPU (s):
0.03944623799998226
GPU speedup over CPU: 71x


In [ ]:
# Test the installation
import os
os.environ['PYTHONPATH'] += ':/content/gdrive/My Drive/computer_vision/models/research/:/content/gdrive/My Drive/computer_vision/models/research/slim/'

!python object_detection/builders/model_builder_tf2_test.py

2021-04-26 03:33:55.094315: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
Running tests under Python 3.7.10: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model
2021-04-26 03:34:37.799149: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-26 03:34:37.800022: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-26 03:34:37.806166: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-26 03:34:37.806702: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB devi

In [ ]:
# Training the model
!python '/content/gdrive/My Drive/computer_vision/workspace/training_demo/model_main_tf2.py' \
        --model_dir='/content/gdrive/My Drive/computer_vision/workspace/training_demo/models/efficientdet_d0_512x512' \
        --pipeline_config_path='/content/gdrive/My Drive/computer_vision/workspace/training_demo/models/efficientdet_d0_512x512/pipeline.config'

2021-04-26 03:36:12.146256: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-26 03:36:26.276103: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-26 03:36:26.276969: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-26 03:36:26.283321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-04-26 03:36:26.283948: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2021-04-26 03:36:26.283989: I tensorflow/stream_executor/platform/default/dso_loade

In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import tensorflow as tf
import datetime
log_dir = '/content/gdrive/My Drive/computer_vision/workspace/training_demo/models/efficientdet_d0_512x512/train' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
# Open TensorBoard
%tensorboard --logdir '/content/gdrive/My Drive/computer_vision/workspace/training_demo/models/efficientdet_d0_512x512/train' --bind_all

In [ ]:
# Export a trained model
!python '/content/gdrive/My Drive/computer_vision/workspace/training_demo/exporter_main_v2.py' \
        --input_type image_tensor --pipeline_config_path '/content/gdrive/My Drive/computer_vision/workspace/training_demo/models/efficientdet_d0_512x512/pipeline.config' \
        --trained_checkpoint_dir '/content/gdrive/My Drive/computer_vision/workspace/training_demo/models/efficientdet_d0_512x512' \
        --output_directory '/content/gdrive/My Drive/computer_vision/workspace/training_demo/exported-models/my_efficientdet_model'